In [2]:
import json
import re
from collections import Counter
import numpy as np
import random
import torch
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
#disable ssl
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('stopwords')

from flyvec_model import FlyvecModel
import preprocess_books as prep
import utils

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


In [4]:
# Load data and combine books into a string
combined_books_text = prep.load_book_data('data/train.json')

# Clean/filter text
words_list, word_counts, vocab = prep.preprocess_text(combined_books_text)

# Create training data: np array shape [N, window_size]
train_data = prep.prepare_training_data(words_list, window_size=10)

print(f'train data shape: {train_data.shape}')
print(f'train sample: {random.choice(train_data)}')

train data shape: (232446, 10)
train sample: ['<unk>' 'made' 'use' 'client' 'imagination' 'help' 'work' 'recognized'
 '<unk>' '<unk>']


In [ ]:
# Create model
model = FlyvecModel(
    K_size=350,            # Number of neurons
    vocab_size=len(vocab),  # Size of vocab
    k=5,                    # Update top-k neurons
    lr=.1,                  # Learning rate
    norm_rate=5,            # Normalization rate
)

# Create encoder
enc = utils.Encoder(vocab)

# Train model
num_epochs = 3

for i in range(num_epochs):
    for num, sample in enumerate(tqdm(train_data, desc=f'Epoch {i+1}/{num_epochs}', ncols=100, leave=True)):        
        enc_sample = enc.one_hot(sample)
        model.update(enc_sample)

# Save model
utils.save_model(model, f'trained_models/context_model_epoch{num_epochs}_books.pt')

In [ ]:
# Find words with embeddings most similar to the target word embedding
target_word = 'ship'
hash_length = 40
top_N_closest = 20

#model = utils.load_model('trained_models/context_model_epoch3_books.pt')

utils.calc_print_sim_words(
    vocab=vocab,
    word_counts=word_counts,
    model=model,
    word=target_word,
    hash_len=hash_length,
    top_N=top_N_closest,
)